# 1. Data preprocessing

Before running this notebook, ensure all data have been downloaded from their respectives sources at a monthly timestep with at least 1x1° resolution across 2003-2018, and updated the local paths accordingly in `CONSTANTS`:

- ERA5:
  - Variables: `t2m`, `tp`, `e`, `pev`, `sic`
  - Source: https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-single-levels-monthly-means
- OC-CCI
  - Variables: `chlor_a`
  - Source: https://www.oceancolour.org/thredds/ncss/CCI_ALL-v6.0-MONTHLY?var=chlor_a&disableLLSubset=on&disableProjSubset=on&horizStride=1&time_start=2003-01-01T00%3A00%3A00Z&time_end=2019-01-01T00%3A00%3A00Z&timeStride=1&addLatLon=true&accept=netcdf
  - Website: https://www.oceancolour.org/browser/index.php?product=chlor_a&page=0&period=monthly&version=6&limit=24&from=2003-01-01&to=2018-12-31
- HadISST SIC:
  - Variables: `sic` 
  - Source: https://www.metoffice.gov.uk/hadobs/hadisst/data/HadISST_ice.nc.gz
  - Website: https://www.metoffice.gov.uk/hadobs/hadisst/data/download.html
- MODIS Terra NDVI:
  - Variables: `CMG_0_05_Deg_Monthly_NDVI`
  - Source: https://e4ftl01.cr.usgs.gov/MOLT/MOD13C2.061/

## Imports + constants

In [1]:
!pip install --no-cache --quiet regionmask 

In [2]:
from dask.diagnostics import ProgressBar
from datetime import timedelta

import cftime
import dask
import numpy as np
import xarray

import libs.utils

In [3]:
CONSTANTS = {
    'path_data_era5': 'data/era5_2003_2018.nc',
    'path_data_chla': 'data/oc-cci-chla/ESACCI-OC-L3S-CHLOR_A-MERGED-1M_MONTHLY_4km_GEO_PML_OCx-*-fv6.0.nc',
    'path_data_sic': 'data/HadISST_ice.nc',
    'path_data_ndvi': 'data/ndvi/MOD13C2.*.nc',
    'year_start': 2003,
    'year_end': 2018,
}

## Functions

In [4]:
def assign_coords_time(data):
    # Extract time from filename
    yyyydd = data.encoding['source'].split('/').pop()
    yyyy = int(yyyydd[9:13])
    dd = int(yyyydd[13:16])

    # Convert to time
    time = cftime.datetime(yyyy, 1, 1, calendar='standard') + timedelta(days=dd)

    data = data.assign_coords({ 'time': time })

    return data


def assign_coords_latlon(
    data,
    dim_lat='lat',
    dim_lon='lon'
):
    data = data\
        .assign_coords({
            dim_lat: -90.0 * (
                (data[dim_lat].values - (max(data[dim_lat].values) / 2.0))
                / (max(data[dim_lat].values) / 2.0)
            ),
            dim_lon: ((360.0 * data[dim_lon].values / max(data[dim_lon].values)) - 180.0)
        })\
        .rename({
            dim_lat: 'lat',
            dim_lon: 'lon'
        })\
        .sortby('lat', 'lon')

    return data


def calc_slice_monthly_climatology(
    data,
    year_start=CONSTANTS['year_start'],
    year_end=CONSTANTS['year_end']
):
    return data\
        .sel(time=slice(f'{year_start}-01-01', f'{year_end}-12-31'))\
        .groupby('time.month')\
        .mean('time')\
        .chunk(dict(month=-1))\
        .compute()

## Main

### Climatological ERA5

In [5]:
with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    era5 = xarray.open_mfdataset(
        engine='netcdf4',
        paths=CONSTANTS['path_data_era5'],
        use_cftime=True
    )

# Standardise naming
era5 = era5.rename({
    'latitude': 'lat',
    'longitude': 'lon',
    't2m': 'tas',
    'tp': 'pr',
    'e': 'evspsbl'
})

# Standardise units
era5['pr'] = era5.pr * 1000.0 / 86400.0
era5['evspsbl'] = era5.evspsbl * -1000.0 / 86400.0
era5['prnet'] = era5.pr - era5.evspsbl
era5['pet'] = libs.utils.mask_to_land(era5.pev * -1000.0 / 86400.0)

# Select time period and calculate monthly climatology
era5_monthly = calc_slice_monthly_climatology(era5)

# Calculate aridity index `ai`, biotemperature `tbio` and net precipitation `prnet`
era5_monthly['ai'] = era5_monthly.pr / (era5_monthly.pr + era5_monthly.pet)
era5_monthly['tbio'] = xarray.where(
    era5_monthly.tas > 303.15,
    303.15,
    xarray.where(
        era5_monthly.tas < 273.15,
        273.15,
        era5_monthly.tas
    )
)
era5_monthly['prnet'] = era5_monthly.pr - era5_monthly.evspsbl

### MARINE: Chlorophyll-a (Chl-a) + SIC

In [6]:
# --- Chlorophyll-a (Chl-a) data ---
chla_ds = xarray.open_mfdataset(
    paths=CONSTANTS['path_data_chla'],
    combine='by_coords'
)

chla_monthly = libs.utils.mask_to_ocean(
    calc_slice_monthly_climatology(chla_ds.chlor_a)
)

In [7]:
# --- Sea ice concentration (SIC) data ---
sic_ds = xarray.open_mfdataset(
    paths=CONSTANTS['path_data_sic'],
    combine='by_coords'
)

sic_ds = sic_ds.rename({
    'latitude': 'lat',
    'longitude': 'lon'
})

sic_monthly = libs.utils.mask_to_ocean(
    calc_slice_monthly_climatology(sic_ds.sic)
)

### TERRESTRIAL: NDVI data

In [8]:
ndvi_evi = xarray.open_mfdataset(
    paths=CONSTANTS['path_data_ndvi'],
    combine='nested',
    concat_dim='time',
    engine='netcdf4',
    preprocess=lambda data: assign_coords_time(
        assign_coords_latlon(
            data,
            dim_lat='YDim_MOD_Grid_monthly_CMG_VI',
            dim_lon='XDim_MOD_Grid_monthly_CMG_VI'
        )
    )
)

with dask.config.set(**{ 'array.slicing.split_large_chunks': True }):
    ndvi_monthly = calc_slice_monthly_climatology(
        ndvi_evi.CMG_0_05_Deg_Monthly_NDVI / 1e8
    )

    # Prevent error of overlapping -180/180 longitudes
    ndvi_monthly = libs.utils.mask_to_land(
        ndvi_monthly.where(ndvi_monthly.lon < 180, drop=True)
    )

### Regridding

In [9]:
target_grid = sic_monthly

era5_monthly_regrid = libs.utils.regrid_data(
    era5_monthly,
    target_grid,
    method='bilinear',
    regrid_kwargs={
        'extrap_method': 'nearest_s2d',
    }
)

chla_monthly_regrid = libs.utils.regrid_data(
    chla_monthly,
    target_grid,
    method='bilinear',
    regrid_kwargs={
        'extrap_method': 'nearest_s2d',
    }
)

ndvi_monthly_regrid = libs.utils.regrid_data(
    ndvi_monthly,
    target_grid,
    method='bilinear',
    regrid_kwargs={
        'extrap_method': 'nearest_s2d',
    }
)

hwloc/linux: Ignoring PCI device with non-16bit domain.
Pass --enable-32bits-pci-domain to configure to support such devices
(warning: it would break the library ABI, don't enable unless really needed).


### Chla high latitude gap-filling/interpolation

In [10]:
# Interpolate chl-a twice to fill seasonal cycle of both hemispheres
chla_monthly_interp = chla_monthly_regrid\
    .interpolate_na('month')\
    .roll(month=6)\
    .interpolate_na('month')\
    .roll(month=6)\
    .compute()

# Weight interpolated values against sea ice concentration
# with a minimum value of 0.01
chla_monthly_interp2 = chla_monthly_interp\
    .where(
        sic_monthly.isnull(),
        np.fmax(chla_monthly_interp * (1.0 - sic_monthly), 0.01)
    )\
    .compute()

### Write preprocessed data

In [11]:
data_obs = xarray.Dataset(
    data_vars={
        'chla': chla_monthly_interp2,
        'ndvi': ndvi_monthly_regrid,
        'sic': sic_monthly,
    },
    coords=chla_monthly_interp2.coords
)

write = data_obs.to_netcdf(
    'data/obs_processed_2003_2018.nc',
    compute=False,
    engine='netcdf4',
    unlimited_dims=['time']
)

with ProgressBar():
    write.compute()

[########################################] | 100% Completed | 101.62 ms


In [12]:
write = era5_monthly_regrid.to_netcdf(
    'data/era5_processed_2003_2018.nc',
    compute=False,
    engine='netcdf4',
    unlimited_dims=['time']
)

with ProgressBar():
    write.compute()

[########################################] | 100% Completed | 101.54 ms
